Performance
===

In [2]:
import pandas as pd
import numpy as np

from scipy.stats import norm
import copy


In [3]:
from trempy.read.read import read
from trempy.clsModel import ModelCls
from trempy.process.process import process
from trempy.shared.shared_auxiliary import dist_class_attributes
from trempy.shared.shared_auxiliary import get_optimal_compensations
from trempy.config_trempy import NEVER_SWITCHERS
from trempy.config_trempy import TINY_FLOAT
from trempy.config_trempy import HUGE_FLOAT


In [4]:
fname = 'performance/start.trempy.ini'
model_obj = ModelCls(fname)

In [5]:
# Prepare data
# Distribute class parameters except for economic parameters and version-specific thing
args = [model_obj, 'version', 'est_file', 'questions', 'paras_obj', 'start', 'cutoffs',
        'maxfun', 'est_detailed', 'opt_options', 'optimizer', 'est_agents', 'num_skip']

version, est_file, questions, paras_obj, start, cutoffs, maxfun, est_detailed, \
    opt_options, optimizer, est_agents, num_skip = dist_class_attributes(*args)

# Handle version-specific objects not included in the para_obj
if version in ['scaled_archimedean']:
    upper, marginals = dist_class_attributes(*[model_obj, 'upper', 'marginals'])
    version_specific = {'upper': upper, 'marginals': marginals}
elif version in ['nonstationary']:
    version_specific = dict()

# We only need to continue if there is at least one parameter to actually estimate.
if len(paras_obj.get_values('optim', 'free')) == 0:
    raise TrempyError('no free parameter to estimate')

# Some initial setup
df_obs = process(est_file, questions, num_skip, est_agents, cutoffs)

Criterium function

In [6]:
# THE ORIGINAL FUNCTION!
def criterion_function(df, questions, cutoffs, paras_obj, version, sds, **version_specific):
    """Calculate the likelihood of the observed sample.

    sds: standard deviations for each question.
    model_obj: the ModelCls object
    version: nonstationary or scaled_archimedean
    """
    # Distribute parameters
    m_optimal = get_optimal_compensations(version=version, paras_obj=paras_obj, questions=questions,
                                          **version_specific)

    contribs = []
    for i, q in enumerate(questions):
        df_subset = df.loc[(slice(None), q), "Compensation"].copy().to_frame('Compensation')
        lower, upper = cutoffs[q]

        is_not = df_subset['Compensation'].between(lower, upper, inclusive=False)
        is_upper = df_subset['Compensation'].isin([NEVER_SWITCHERS])
        is_lower = df_subset['Compensation'].isin([lower])

        rv = norm(loc=m_optimal[q], scale=sds[i])

        # Calculate likelihoods
        df_subset['likl_not'] = np.nan
        df_subset['likl_not'] = df_subset['likl_not'].mask(~is_not)

        df_subset['likl_not'].loc[is_not, :] = rv.pdf(df_subset['Compensation'].loc[is_not, :])
        df_subset['likl_upper'] = 1.0 - rv.cdf(upper)
        df_subset['likl_lower'] = rv.cdf(lower)

        df_subset['likl'] = 0.0
        df_subset['likl'][is_upper] = df_subset['likl_upper'].loc[is_upper]
        df_subset['likl'][is_lower] = df_subset['likl_lower'].loc[is_lower]
        df_subset['likl'][is_not] = df_subset['likl_not'].loc[is_not]

        contribs += df_subset['likl'].values.tolist()

    rslt = -np.mean(np.log(np.clip(sorted(contribs), TINY_FLOAT, np.inf)))

    return rslt

In [16]:
def criterion_function_2(df, questions, cutoffs, paras_obj, version, sds, **version_specific):
    """Calculate the likelihood of the observed sample.

    sds: standard deviations for each question.
    model_obj: the ModelCls object
    version: nonstationary or scaled_archimedean
    """
    # Distribute parameters
    m_optimal = get_optimal_compensations(version=version, paras_obj=paras_obj, questions=questions,
                                          **version_specific)

    contribs = []
    for i, q in enumerate(questions):
        df_subset = df.loc[(slice(None), q), "Compensation"].copy().to_frame('Compensation')
        lower, upper = cutoffs[q]

        is_not = df_subset['Compensation'].between(lower, upper, inclusive=False)
        is_upper = df_subset['Compensation'].isin([NEVER_SWITCHERS])
        is_lower = ~(is_not | is_upper)

        rv = norm(loc=m_optimal[q], scale=sds[i])

        # Calculate likelihoods
        df_subset['likl_not'] = rv.pdf(df_subset['Compensation'])
        df_subset['likl_upper'] = 1.0 - rv.cdf(upper)
        df_subset['likl_lower'] = rv.cdf(lower)

        
        likl_upper = df_subset['likl_upper'].loc[is_upper, :]
        likl_lower = df_subset['likl_lower'].loc[is_lower, :]
        likl_interior = df_subset['likl_not'].loc[is_not, :]

        contribs += likl_upper.values.tolist()
        contribs += likl_lower.values.tolist()
        contribs += likl_interior.values.tolist()

    rslt = -np.mean(np.log(np.clip(sorted(contribs), TINY_FLOAT, np.inf)))

    return rslt

In [9]:
# Construct relevant set of parameters
x_econ_all_current = paras_obj.get_values('econ', 'all')

# Get standard deviations. They have a larger index than nparas_econ.
nparas_econ = paras_obj.attr['nparas_econ']
stands = x_econ_all_current[nparas_econ:]

In [18]:
%timeit criterion_function(df_obs, questions, cutoffs, paras_obj, version, stands, **version_specific)

2.39 s ± 83.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
criterion_function(df_obs, questions, cutoffs, paras_obj, version, stands, **version_specific)

9.531515655563092

In [17]:
criterion_function_2(df_obs, questions, cutoffs, paras_obj, version, stands, **version_specific)

8.554537304763064

Improvements
===

In [19]:
def criterion_function_new(df, questions, cutoffs, paras_obj, version, sds, **version_specific):
    """Calculate the likelihood of the observed sample."""

    m_optimal = get_optimal_compensations(
        version=version, paras_obj=paras_obj, questions=questions, **version_specific)
    data = copy.deepcopy(df)
    
    # Add cutoff column
    df_cutoff = pd.DataFrame.from_dict(cutoffs, orient='index', columns=['lower', 'upper'])
    df_cutoff = df_cutoff.reset_index()
    df_cutoff = df_cutoff.rename(columns={'index': 'Question'})
    data = data.reset_index()
    data = data.merge(df_cutoff, how='left', on='Question')

    # Add indicator for interior/boundary choices.
    data['is_interior'] = (data.lower < data.Compensation) & (data.Compensation < data.upper)
    data['is_upper'] = data['Compensation'].isin([NEVER_SWITCHERS])
    data['is_lower'] = ~(data.is_interior | data.is_upper)

    # Add column with the theortically optimal decision for each question
    df_m_optimal = pd.DataFrame.from_dict(m_optimal, orient='index', columns=['m_optim'])
    df_m_optimal = df_m_optimal.reset_index().rename(columns={'index': 'Question'})
    data = data.merge(df_m_optimal, how='left', on='Question')

    # Add column with standard deviation of the implementation error for each question
    df_std = pd.DataFrame(sds, index=questions, columns=['std'])
    df_std = df_std.reset_index().rename(columns={'index': 'Question'})
    data = data.merge(df_std, how='left', on='Question')
    
    # Debugging. Delete later.
    np.testing.assert_equal(data.isna().sum().sum() == 0, True)
    np.testing.assert_equal(data.shape[0] == df.shape[0], True)
    
    # Add standardized choices
    data['choice_standardized'] = (data['Compensation'] - data['m_optim']) / data['std']
    data['lower_standardized'] = (data['lower'] - data['m_optim']) / data['std']
    data['upper_standardized'] = (data['upper'] - data['m_optim']) / data['std']

    # We only need the standard normal distribution for standardized choices.
    rv = norm(loc=0.0, scale=1.0)

    # Compute likelihood of each question-subject pair based on interior/boundary status.
    likl_interior = (rv.pdf(data['choice_standardized'].loc[data['is_interior']]) /
                     data['std'].loc[data['is_interior']])
    
    likl_upper = 1.0 - rv.cdf(data['upper_standardized'].loc[data['is_upper']])
    likl_lower = rv.cdf(data['lower_standardized'].loc[data['is_lower']])
    
    # Add all summands to one list.
    contribs = likl_interior.tolist() + likl_lower.tolist() + likl_upper.tolist()
        
    # Compute the average negative log-likelihood
    rslt = - np.mean(np.log(np.clip(sorted(contribs), TINY_FLOAT, np.inf)))

    return rslt

In [20]:
criterion_function_new(df_obs, questions, cutoffs, paras_obj, version, stands, **version_specific)

8.554537304763064

In [21]:
%timeit criterion_function_new(df_obs, questions, cutoffs, paras_obj, version, stands, **version_specific)

370 ms ± 17.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
